# FLOWCELL | OPERATION

[2024-05-09] Suivi des opérations. Renseigner les noms de la _flowcell_ et de l'exérience (informations dans la SampleSheet, communiquées par courriel du CQGC ou récupérées de BaseSpace).

0. Mise-en-place de l'environnement de travail
1. Collecter les informations sur les familles et les métriques DragenGermline
2. Téléverser les FASTQs sur BaseSpace
3. Créer les cas sur Emedgene 
    1. Générer le fichier "emg_batch_manifest.csv" `emg_make_batch_from_nanuq.py ${FC}`
    2. Glisser-déposer dans Emedgene le fichier "emg_batch_manifest.csv"
4. Archiver les résultats
5. Nettoyage


## TROUBLESHOOTING

- Afin de connecter les informations génétiques des familles (issues de Nanuq) aux phénotypes (termes HPOs contenus dans Phenotips), il est impératif que les deux champs EP (Établissement Public) et MRN (Medical Record Number) soient bien renseignés dans les deux systèmes par les collègues. Sinon, il faut obtenir les informations de Phenotips (identifiants PID) par "Camille Varin-Tremblay (HSJ)" <camille.varin-tremblay.hsj@ssss.gouv.qc.ca>.
- Si le séquençage est réalisé avec le NovaSeq6000, il ne sera malheureusement pas possible de récupérer les métriques avant de créer les cas dans Emedgene. Dans ce cas, il faut s'assurer que le labo a demandé au NovaSeq6000 de faire la déconvolution sur BaseSpace et passer à l'étape 3, "créer les cas sur Emedgene", une fois que les FASTQs ont été générés (suivi sur l'onglet [Analyses](https://chusj.cac1.sh.basespace.illumina.com/analyses)). 
- Dans la situation où la déconvolution n'a pas lieu sur BaseSpace, il faut déconvoluer les BCLs avec les serveurs DRAGEN du CQGC et envoyer les FASTQs résultants sur BaseSpace (passer à l'étape 2, "Téléverser les FASTQs sur BaseSpace").


In [ ]:
## 0. Mise en place de l'environnement de travail
# ssh ${USER}@10.128.80.26 # Renseigner la valeur de ${FC}

screen -S prag
conda activate CQGC-utils

export FC=""
a=($(echo ${FC} | tr '_' '\n'))
export FC_SHORT="${a[1]}_${a[2]}"
export BASEDIR="/mnt/spxp-app02/staging/hiseq_raw/${a[1]}"
export WORKDIR="/mnt/spxp-app02/staging2/dragen"


In [ ]:
## 1. Collecter les metriques

mkdir ${WORKDIR}/${FC_SHORT} && cd "$_"
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_dragen_metrics.py ${FC}

Récupérer les métriques et le fichier "samples_list.csv" générés par la commande ci-dessus. 

```PowerShell:
$FC_SHORT=""
Set-Location D:\HSJ\Projects\PRAGMatIQ\Runs\$FC_SHORT
scp hienle@10.128.80.26:/mnt/spxp-app02/staging2/dragen/$FC_SHORT/* .
```

Envoyer les fichiers `${FC_SHORT}_metrics.csv` et `${FC_SHORT}_metrics.html` dans un courriel aux personnes responsables:

- "Dr Jacques Michaud (HSJ)" <jacques.michaud.med@ssss.gouv.qc.ca>
- "Camille Varin-Tremblay (HSJ)" <camille.varin-tremblay.hsj@ssss.gouv.qc.ca>
- "Rene Allard (HSJ)" <rene.allard.hsj@ssss.gouv.qc.ca>

Si les métriques passent les critères d'acceptabilité, procéder au téléversement des FASTQs dans BaseSpace. Sinon, attendre la réponse des responsables et, s'il y a lieu, supprimer les échantillons de mauvaise qualité de `samples_list.csv` avant de procéder.


In [ ]:
## 2. Upload FASTQ sur BaseSpace

cd ${WORKDIR}/${FC_SHORT}
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_upload_fastqs.py

In [ ]:
## 3. Créer les cas sur Emedgene 
###  1. Générer le fichier "emg_batch_manifest.csv" `emg_make_batch_from_nanuq.py ${FC}`
###  2. Glisser-déposer dans Emedgene le fichier "emg_batch_manifest.csv"

cd ${WORKDIR}
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_make_batch_from_nanuq.py ${FC_SHORT} 2>&1 | tee ${FC_SHORT}/emg_make_batch.log

In [ ]:
## 4. Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done

In [ ]:
# 5. Nettoyage
# Avant d'effacer le répertoire de la run, otifier le laboratoire qui doit sauvegarder quelqeues métriques

`rm -R ${BASEDIR}/${FC}`

## Logs